In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import NearestNeighbors
import joblib

In [5]:
file_path = "gaddi.xls"  # Update path if needed
df = pd.read_excel(file_path)

# 🔹 Step 2: Data Preprocessing
df.dropna(inplace=True)

In [6]:
df

,Brand,Model,Year,Engine_Size,Fuel_Type,Transmission,Mileage,Doors,Owner_Count,Price
0,Kia,Rio,2020,4.2,Diesel,Manual,289944,3,5,8501
1,Chevrolet,Malibu,2012,2.0,Hybrid,Automatic,5356,2,3,12092
2,Mercedes,GLA,2020,4.2,Diesel,Automatic,231440,4,2,11171
3,Audi,Q5,2023,2.0,Electric,Manual,160971,2,1,11780
4,Volkswagen,Golf,2003,2.6,Hybrid,Semi-Automatic,286618,3,3,2867
...,...,...,...,...,...,...,...,...,...,...
9995,Kia,Optima,2004,3.7,Diesel,Semi-Automatic,5794,2,4,8884
9996,Chevrolet,Impala,2002,1.4,Electric,Automatic,168000,2,1,6240
9997,BMW,3 Series,2010,3.0,Petrol,Automatic,86664,5,1,9866
9998,Ford,Explorer,2002,1.4,Hybrid,Automatic,225772,4,1,4084


In [7]:
# label_encoders = {}
# categorical_columns = ["Brand", "Fuel_Type", "Transmission", "Mileage"]
# Define Feature Columns
# feature_cols = ["Price", "Fuel_Type", "Transmission", "Mileage", "Engine_Size"]  # Modify if needed
# categorical_cols = ["Fuel_Type", "Transmission"] 
categorical_cols = ["Fuel_Type", "Transmission"]
numerical_cols = ["Price", "Mileage", "Engine_Size"]

In [8]:
label_encoders = {}


In [9]:
# for col in categorical_columns:
#     le = LabelEncoder()
#     df[col] = le.fit_transform(df[col])
#     label_encoders[col] = le 
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])  # Convert to numerical labels
    label_encoders[col] = le 

In [10]:
features = ["Price", "Fuel_Type", "Transmission", "Mileage", "Engine_Size"]
X = df[features]

In [ ]:
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)
# scaler = StandardScaler()
# feature_cols = ["Price", "Fuel_Type", "Transmission", "Mileage", "Engine_Size"]  # Adjust based on your dataset
# df_scaled = scaler.fit_transform(df[feature_cols])

# # Save Scaler (run once after fitting)
# joblib.dump(scaler, "scaler.pkl")

# # Load Scaler when using Streamlit
# scaler = joblib.load("scaler.pkl")

joblib.dump(label_encoders, "label_encoders.pkl")

# Initialize and Fit Scaler
# scaler = StandardScaler()
# df_scaled = scaler.fit_transform(df[feature_cols])
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df[numerical_cols + categorical_cols])  

# Save the Scaler
joblib.dump(scaler, "scaler.pkl")
joblib.dump(label_encoders, "label_encoders.pkl")
# joblib.dump(scaler, "scaler.pkl")

# print("✅ Label Encoders and Scaler have been saved.")
scaler = joblib.load("scaler.pkl")
label_encoders = joblib.load("label_encoders.pkl")


['label_encoders.pkl']

In [12]:
knn_model = NearestNeighbors(n_neighbors=5, metric="euclidean")
knn_model.fit(X_scaled)

NameError: name 'X_scaled' is not defined

In [13]:
def recommend_car(price, fuel, transmission, mileage, engine):
    try:
        # Convert Fuel Type & Transmission from string to encoded value
        fuel_mapping = dict(zip(label_encoders["Fuel_Type"].inverse_transform(range(len(label_encoders["Fuel_Type"].classes_))),
                                range(len(label_encoders["Fuel_Type"].classes_))))
        transmission_mapping = dict(zip(label_encoders["Transmission"].inverse_transform(range(len(label_encoders["Transmission"].classes_))),
                                        range(len(label_encoders["Transmission"].classes_))))

        # Check if user input exists in mapping
        if fuel not in fuel_mapping:
            raise ValueError(f"Fuel Type '{fuel}' not found. Available: {list(fuel_mapping.keys())}")

        if transmission not in transmission_mapping:
            raise ValueError(f"Transmission '{transmission}' not found. Available: {list(transmission_mapping.keys())}")

        # Encode user input
        fuel_encoded = fuel_mapping[fuel]
        transmission_encoded = transmission_mapping[transmission]

        # Create input array (ensure it has the same features as X)
        user_input = np.array([[price, fuel_encoded, transmission_encoded, mileage, engine]])
        user_input_scaled = scaler.transform(user_input)  # Scale input

        # Find nearest neighbors
        distances, indices = knn_model.kneighbors(user_input_scaled)

        # Get recommended cars
        recommended_cars = df.iloc[indices[0]][["Brand", "Model", "Price"]]

        # Decode Brand names
        recommended_cars["Brand"] = label_encoders["Brand"].inverse_transform(recommended_cars["Brand"])

        return recommended_cars

    except Exception as e:
        return f"Error: {e}"


In [ ]:
user_price = 15  # ₹ in Lakh
user_fuel = "Petrol"
user_transmission = "Automatic"
user_mileage = 18
user_engine = 1498
user_car_type = 1
# user_price = float(input("Enter your Budget (in Lakh ₹): "))
# user_fuel = input("Enter Fuel Type (Diesel/Petrol/CNG/Electric): ")
# user_transmission = input("Enter Transmission (Manual/Automatic): ")
# user_mileage = float(input("Enter Mileage (km/l): "))
# user_engine = int(input("Enter Engine Power (cc): "))


# Encode categorical inputs
try:
    fuel_encoded = label_encoders["Fuel_Type"].transform([user_fuel])[0]
    transmission_encoded = label_encoders["Transmission"].transform([user_transmission])[0]
except ValueError:
    print("❌ Invalid Fuel Type or Transmission! Please check dataset values.")
    exit()

# Create input array & scale it
user_input = np.array([[user_price, fuel_encoded, transmission_encoded, user_mileage, user_engine]])
user_input_scaled = scaler.transform(user_input)

c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [15]:
print("\n🔹 Recommended Cars:")
print(recommend_car(user_price, user_fuel, user_transmission, user_mileage, user_engine))  # Removed user_car_type



🔹 Recommended Cars:
Error: This NearestNeighbors instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.


c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
